### Q1: What to do while the data is away?

In [2]:
108*10**(-3)/(0.89*11/(2*10**9))

22063329.928498466

### Q2: Fibonacci... again.

#### Part 1: Do it.

In [3]:
%%file fibonacci.c
#include <stdio.h>

int main(int argc, char** argv)
{
    printf("fib(8) = %d",fib(8));
    return 0;
}

int fib(int n)
{
  if (n==1)
    return 0;
  if (n==2)
    return 1;
  else
    return (fib(n-1)+fib(n-2));
}


Writing fibonacci.c


In [4]:
!gcc -o fibonacci fibonacci.c

In [5]:
!fibonacci

fib(8) = 13


#### Part 2: Inspecting the stack

In [7]:
!gcc -g -o fibonacci fibonacci.c

There were 7 stack frames (including main). The current value of $n$ is 2.

### Q3: Crafting a buffer overflow attack

#### Part 1: Get familiar

In [1]:
%%file overflow.c
// overflow.c
#include <stdlib.h>
#include <stdio.h>

void win()
{
  printf("Success!\n");
  exit(0);
}

int victim_function(int index)
{
  int64_t a[5] = {1,2,3,4,5};
  a[index] = (int64_t)(&win);
  return 0;
}

int main(int argc, char **argv)
{
  int magic_number = 0;

  if( argc>1 ) {
    magic_number = atoi(argv[1]);
  } else {
    printf("Please enter a magic number.\n");
    return -1;
  }

  victim_function( magic_number );

  return 0;
}

Writing overflow.c


#### Part 2: Staking out the target

(gdb) print/x a[0]@5

{0x1, 0x2, 0x3, 0x4, 0x5}

(gdb) next

(gdb) print/x a[0]@5

{0x1004010e0, 0x2, 0x3, 0x4, 0x5}


#### Part 3: Springing the trap

return address: {0x000000010040118f}

The value of the offset relative to the start of the array is 7. The other function on the stack has name ?? and we get the error: "Backtrace stopped: previous frame inner to this frame (corrupt stack?)". I think the call stack looks like this because win is never explicitly called from a function rather it is called because we replaced a return address with its address. Therefore, the name of the function it was called from appears as unknown.